In [12]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  
from IPython.display import HTML

In [13]:
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, precision_score, recall_score
import pandas
import numpy as np
import papermill as pm
import json
import matplotlib.pyplot as plt
import os
import uuid
from db import Result
import ast
import math
import scrapbook as sb
pandas.options.display.float_format = '{:,.3f}'.format
import pickle
from clinical_data_models import features_data

In [14]:
from evaluate import plot_learning_curve, plot_accuracy_curve, load, get_results, get_labels, transform_binary_probabilities, transform_binary_predictions, calculate_accuracy_loss, plot_confusion_matrix, plot_precision_recall, plot_roc_curve, calculate_pr_auc, calculate_confusion_matrix_stats, calculate_confusion_matrix, plot_precision_recall#, plot_tsne, plot_multiple_grad_cam
from data_gen import data
from run import characterize_data
from config import config

In [15]:
# T1C results
t1c_results = Result.query.filter(Result.description == "t1c-t2-exp", Result.input_form == "t1c").all()
#t1_results = Result.query.filter(Result.description == "t1_final_200").all()

# T1 results
t1_results = Result.query.filter(Result.description == "t1_20210404", Result.input_form == "t1").all()
#t1_results = Result.query.filter(Result.description == "t1_final_200").all()

# T2 results
t2_results = Result.query.filter(Result.description == "t1c-t2-exp", Result.input_form == "t2").all()
#t2_results = Result.query.filter(Result.description == "t2_final_run_200").all()

# Clinical results
uuids = ["ea82b20c-ced8-4787-91f9-5eb3a15f5554",
"4af80ba2-96de-4ca4-a2e2-f003c419bb02",
"7b22fa31-5787-4fe1-a1a3-03f2385e2324",
"f617e4d1-068d-4c19-b864-bf4d61f11ab5",
"a1fd5687-622a-44c0-ae4f-6bdaab289b1e",
"228e571d-cd25-4bae-9983-7043d2294882",
"51d7476d-7445-4a85-9a52-38bf06d63b13",
"0c1a57f3-7183-42bf-92dc-28c278d150ac",
"4a7db6a9-f55e-4719-8f60-460e1dc7a697",
"6860ca52-0f15-4385-990d-0cb64fbfc18c",]
clinical_results = Result.query.filter(Result.uuid.in_(uuids)).all()
#t2_results = Result.query.filter(Result.description == "t2_final_run_200").all()

In [16]:
print(t1_results)
print(t1c_results)
print(t2_results)
print(clinical_results)

[<Result accuracy: 0.6701030731201172>, <Result accuracy: 0.6804123520851135>, <Result accuracy: 0.6907216310501099>, <Result accuracy: 0.6597937941551208>, <Result accuracy: 0.6597937941551208>, <Result accuracy: 0.6082473993301392>, <Result accuracy: 0.6701030731201172>, <Result accuracy: 0.5051546096801758>, <Result accuracy: 0.6907216310501099>, <Result accuracy: 0.6494845151901245>]
[<Result accuracy: 0.5257731676101685>, <Result accuracy: 0.5154638886451721>, <Result accuracy: 0.5154638886451721>, <Result accuracy: 0.5463917255401611>, <Result accuracy: 0.4845360815525055>, <Result accuracy: 0.5154638886451721>, <Result accuracy: 0.5360824465751648>, <Result accuracy: 0.47422680258750916>, <Result accuracy: 0.5154638886451721>, <Result accuracy: 0.5154638886451721>]
[<Result accuracy: 0.7010309100151062>, <Result accuracy: 0.6391752362251282>, <Result accuracy: 0.7010309100151062>, <Result accuracy: 0.6804123520851135>, <Result accuracy: 0.6804123520851135>, <Result accuracy: 0.6

In [17]:
train, validation, test, external = data(seed=uuid.UUID("84a64c17-fe3e-440c-aaaf-e1bd5b02576f"), label_form="outcome_pos", input_form="t1", train_shuffle=False, test_shuffle=False, validation_shuffle=False, train_augment=False, validation_augment=False, test_augment=False)
all_results_t1 = {}
for result in t1_results:
    MODEL = "{}/models/{}-{}.h5".format(config.OUTPUT, result.uuid, result.model)
    model = load(MODEL)
    results = get_results(model, validation)
    labels = get_labels(validation)
    test_results = get_results(model, test)
    test_labels = get_labels(test)
    stats = calculate_confusion_matrix_stats(labels, results)
    #all_results_t1[str(result.uuid)] = {"sensitivity": stats["TPR"][1], "specificity": stats["TNR"][1], "roc_auc": roc_auc_score(labels,results)}
    all_results_t1[str(result.uuid)] = {"roc_auc_val": roc_auc_score(labels,results), "roc_auc_test": roc_auc_score(test_labels,test_results)}

train, validation, test, external = data(seed=uuid.UUID("84a64c17-fe3e-440c-aaaf-e1bd5b02576f"), label_form="outcome_pos", input_form="t1c", train_shuffle=False, test_shuffle=False, validation_shuffle=False, train_augment=False, validation_augment=False, test_augment=False)
all_results_t1c = {}
for result in t1c_results:
    MODEL = "{}/models/{}-{}.h5".format(config.OUTPUT, result.uuid, result.model)
    model = load(MODEL)
    results = get_results(model, validation)
    labels = get_labels(validation)
    test_results = get_results(model, test)
    test_labels = get_labels(test)
    stats = calculate_confusion_matrix_stats(labels, results)
    #all_results_t1c[str(result.uuid)] = {"sensitivity": stats["TPR"][1], "specificity": stats["TNR"][1], "roc_auc": roc_auc_score(labels,results)}
    all_results_t1c[str(result.uuid)] = {"roc_auc_val": roc_auc_score(labels,results), "roc_auc_test": roc_auc_score(test_labels,test_results)}
    

train, validation, test, external = data(seed=uuid.UUID("84a64c17-fe3e-440c-aaaf-e1bd5b02576f"), label_form="outcome_pos", input_form="t2", train_shuffle=False, test_shuffle=False, validation_shuffle=False, train_augment=False, validation_augment=False, test_augment=False)
all_results_t2 = {}
for result in t2_results:
    MODEL = "{}/models/{}-{}.h5".format(config.OUTPUT, result.uuid, result.model)
    model = load(MODEL)
    results = get_results(model, validation)
    labels = get_labels(validation)
    test_results = get_results(model, test)
    test_labels = get_labels(test)
    stats = calculate_confusion_matrix_stats(labels, results)
    #all_results_t2[str(result.uuid)] = {"sensitivity": stats["TPR"][1], "specificity": stats["TNR"][1], "roc_auc": roc_auc_score(labels,results)}
    all_results_t2[str(result.uuid)] = {"roc_auc_val": roc_auc_score(labels,results), "roc_auc_test": roc_auc_score(test_labels,test_results)}

train, validation, test, external = data(seed=uuid.UUID("84a64c17-fe3e-440c-aaaf-e1bd5b02576f"), label_form="outcome_pos", input_form="features", train_shuffle=False, test_shuffle=False, validation_shuffle=False, train_augment=False, validation_augment=False, test_augment=False)
train_set, train_labels, val_set, val_labels, test_set, test_labels, external_set, external_labels = features_data(train, validation, test, external)
all_results_clinical = {}
for result in clinical_results:
    MODEL = "{}/models/{}_features.sav".format(config.OUTPUT, result.uuid)
    model = pickle.load(open(MODEL, 'rb'))
    results = model.predict(val_set).tolist()
    probabilities=model.predict_proba(val_set).tolist()
    labels = val_labels
    test_results = model.predict(test_set).tolist()
    test_labels = test_labels
    #stats = calculate_confusion_matrix_stats(labels, probabilities)
    #all_results_clinical[str(result.uuid)] = {"sensitivity": stats["TPR"][1], "specificity": stats["TNR"][1], "roc_auc": roc_auc_score(labels,results)}
    all_results_clinical[str(result.uuid)] = {"roc_auc_val": roc_auc_score(labels,results), "roc_auc_test": roc_auc_score(test_labels,test_results)}

  0%|          | 0/678 [00:00<?, ?it/s]

                outcome_pos  outcome_neg  outcome_3    age  sex  \
patientID                                                         
bone-brown-1              0            0          0 52.000    1   
bone-brown-10             0            0          0 52.000    0   
bone-brown-100            1            1          2 22.000    1   
bone-brown-101            0            0          0 88.000    0   
bone-brown-102            0            0          0 56.000    1   
...                     ...          ...        ...    ...  ...   
bone-penn-650             1            1          2 20.000    1   
bone-penn-651             0            0          0 16.000    1   
bone-penn-652             1            1          2  8.000    0   
bone-penn-653             1            1          2 16.000    1   
bone-penn-654             1            1          2  3.000    1   

                                                         location        sort  
patientID                                       

  0%|          | 1/678 [00:00<06:29,  1.74it/s]

bone-brown-100
I'm doing the thing.


  0%|          | 2/678 [00:00<04:56,  2.28it/s]

...

 81%|████████▏ | 79/97 [00:01<00:00, 51.33it/s]

bone-china-342
I'm doing the thing.

################################################################################
Exception occurred for: outcome_pos                                                    0
outcome_neg                                                    0
outcome_3                                                      0
age                                                         10.0
sex                                                            0
location       [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
sort                                                    external
Name: bone-china-342, dtype: object
[Errno 2] No such file or directory: 'data_dir/preprocessed/bone-china-342-t1c-imagingVolume.nrrd'
T1C image unavailable
Traceback (most recent call last):
  File "/home/ubuntu/Bone-MRI/data_gen.py", line 480, in generate_from_features
    t1c_masked = load_image(t1c_image_file, t1c_seg_file, verbose=verbose)
  File "/home/ubuntu/Bone-MRI/data_gen.py", line 351, in

100%|██████████| 97/97 [00:01<00:00, 56.85it/s]

bone-china-354
I'm doing the thing.

################################################################################
Exception occurred for: outcome_pos                                                    0
outcome_neg                                                    0
outcome_3                                                      0
age                                                         30.0
sex                                                            0
location       [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
sort                                                    external
Name: bone-china-354, dtype: object
[Errno 2] No such file or directory: 'data_dir/preprocessed/bone-china-354-t1c-imagingVolume.nrrd'
T1C image unavailable
Traceback (most recent call last):
  File "/home/ubuntu/Bone-MRI/data_gen.py", line 480, in generate_from_features
    t1c_masked = load_image(t1c_image_file, t1c_seg_file, verbose=verbose)
  File "/home/ubuntu/Bone-MRI/data_gen.py", line 351, in


/home/ubuntu/Bone-MRI/data_gen.py:187: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.features = np.array(features)


In [21]:
import pandas as pd
print("T1")
print(pd.DataFrame.from_dict(all_results_t1, orient='index'))
print("T1C")
print(pd.DataFrame.from_dict(all_results_t1c, orient='index'))
print("T2")
print(pd.DataFrame.from_dict(all_results_t2, orient='index'))
print("Clinical")
print(pd.DataFrame.from_dict(all_results_clinical, orient='index'))

T1
                                      roc_auc_val  roc_auc_test
1a51c33e-3a08-4f87-a765-73c7f03c9dbb        0.737         0.662
7bf6a93f-e5d3-4c56-9100-0e5168b8170e        0.731         0.609
693740b9-1275-4a30-a69c-1dc84312ca8c        0.755         0.592
98e61e88-67ac-47b4-845a-71595e4e1255        0.773         0.636
7b9381da-29e0-437a-9c26-d2f68e82788f        0.700         0.600
bb2c0dff-ccf6-45ff-871c-2c984d24ffe1        0.671         0.593
ee6d8f14-d9f2-4222-ab3b-1035a61abf51        0.717         0.559
47a27fb8-cab6-430f-bbe8-e837269bcfbe        0.604         0.514
149707e7-1069-424b-b498-a6bfdee6e9d4        0.695         0.559
696397c9-e5c1-4c75-a130-d35b2341a684        0.665         0.630
T1C
                                      roc_auc_val  roc_auc_test
ac00a260-affc-4846-bb4c-a591e4332728        0.645         0.680
6bb799dc-f454-423a-8dbc-37d18159b2df        0.523         0.596
26e0bfee-eadc-4da0-ad8e-e8124a2ebd52        0.572         0.615
c26d1754-30e7-482d-af0c-93834c484

In [20]:
import math
import statistics
def adjusted_wald(p, n, z=1.96):
    p_adj = (n * p + (z**2)/2)/(n+z**2)
    n_adj = n + z**2
    span = z * math.sqrt(p_adj*(1-p_adj)/n_adj)
    return max(0, p_adj - span), min(p_adj + span, 1.0)

avg_t1_val = statistics.mean([all_results_t1[data]["roc_auc_val"] for data in all_results_t1])
avg_t1_test = statistics.mean([all_results_t1[data]["roc_auc_test"] for data in all_results_t1])
ci_val = adjusted_wald(avg_t1_val, 10)
ci_test = adjusted_wald(avg_t1_test, 10)
print("T1 val")
print("{:.2f} ({:.2f}-{:.2f})".format(avg_t1_val, *ci_val))
print("T1 test")
print("{:.2f} ({:.2f}-{:.2f})".format(avg_t1_test, *ci_test))

avg_t1c_val = statistics.mean([all_results_t1c[data]["roc_auc_val"] for data in all_results_t1c])
avg_t1c_test = statistics.mean([all_results_t1c[data]["roc_auc_test"] for data in all_results_t1c])
ci_val = adjusted_wald(avg_t1c_val, 10)
ci_test = adjusted_wald(avg_t1c_test, 10)
print("T1C val")
print("{:.2f} ({:.2f}-{:.2f})".format(avg_t1c_val, *ci_val))
print("T1C test")
print("{:.2f} ({:.2f}-{:.2f})".format(avg_t1c_test, *ci_test))

avg_t2_val = statistics.mean([all_results_t2[data]["roc_auc_val"] for data in all_results_t2])
avg_t2_test = statistics.mean([all_results_t2[data]["roc_auc_test"] for data in all_results_t2])
ci_val = adjusted_wald(avg_t2_val, 10)
ci_test = adjusted_wald(avg_t2_test, 10)
print("T2 val")
print("{:.2f} ({:.2f}-{:.2f})".format(avg_t2_val, *ci_val))
print("T2 test")
print("{:.2f} ({:.2f}-{:.2f})".format(avg_t2_test, *ci_test))

avg_clinical_val = statistics.mean([all_results_clinical[data]["roc_auc_val"] for data in all_results_clinical])
avg_clinical_test = statistics.mean([all_results_clinical[data]["roc_auc_test"] for data in all_results_clinical])
ci_val = adjusted_wald(avg_clinical_val, 10)
ci_test = adjusted_wald(avg_clinical_test, 10)
print("Clinical val")
print("{:.2f} ({:.2f}-{:.2f})".format(avg_clinical_val, *ci_val))
print("Clinical test")
print("{:.2f} ({:.2f}-{:.2f})".format(avg_clinical_test, *ci_test))

T1 val
0.70 (0.40-0.90)
T1 test
0.60 (0.31-0.83)
T1C val
0.58 (0.30-0.82)
T1C test
0.57 (0.29-0.82)
T2 val
0.72 (0.41-0.91)
T2 test
0.66 (0.36-0.87)
Clinical val
0.59 (0.30-0.83)
Clinical test
0.67 (0.36-0.88)


In [23]:
import numpy as np
import math
lists = [all_results_t2, all_results_t1c, all_results_clinical, all_results_t1]
for result in lists:
    val_mean = np.mean([result[data]["roc_auc_val"] for data in result])
    val_std = np.std([result[data]["roc_auc_val"] for data in result])
    test_mean = np.mean([result[data]["roc_auc_test"] for data in result])
    test_std = np.std([result[data]["roc_auc_test"] for data in result])
    print("{:.3f} ({:.3f}-{:.3f})".format(val_mean, val_mean-2.262*val_std/math.sqrt(10), val_mean+2.262*val_std/math.sqrt(10)))
    print("{:.3f} ({:.3f}-{:.3f})".format(test_mean, test_mean-2.262*test_std/math.sqrt(10), test_mean+2.262*test_std/math.sqrt(10)))

0.722 (0.696-0.748)
0.662 (0.620-0.705)
0.579 (0.536-0.621)
0.574 (0.522-0.626)
0.591 (0.591-0.591)
0.667 (0.667-0.667)
0.705 (0.671-0.738)
0.596 (0.566-0.625)
